In [1]:
import pandas as pd, numpy as np

In [106]:
#Load PIMMART data
pim_df = pd.read_csv('PIM_Data_New_50_82Mn.zip')

In [109]:
pim_df.columns

Index(['ITM_ID', 'GTIN_NO', 'GTIN_NAM', 'PMY_DPT_CD', 'PMY_DPT_DSC',
       'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC',
       'SUBCOM_CD', 'SUBCOM_DSC', 'ITM_LVL_DSC', 'VND_ECOM_DSC'],
      dtype='object')

In [110]:
pim_load = pd.read_csv('PIM_DATA_WITH_FLAVOR_ATTB_100323.csv')
pim_load = pim_load.merge(pim_df, how = 'right', on = list(pim_df.columns))

In [112]:
pim_load['Flavor_Flag'] = pim_load.Flavor_Code.fillna('Null').apply(lambda x: 'Y' if x != 'Null'  else 'N')

In [113]:
#Exclude SUBCOMs with no flavor flag avilable
pim_load = pim_load[pim_load.Flavor_Code != 'NONE']
flavor_subcom_cds = pim_load[pim_load.Flavor_Flag == 'Y'].SUBCOM_CD.unique()
pim_load = pim_load[pim_load.SUBCOM_CD.isin(flavor_subcom_cds)]

In [114]:
pim_load.shape

(944931, 18)

In [115]:
pim_load.Flavor_Flag.value_counts()

N    913725
Y     31206
Name: Flavor_Flag, dtype: int64

In [103]:
flavor_df = pd.DataFrame(pim_load.groupby('SUBCOM_DSC').apply(lambda x:dict( x.Flavor_Flag.value_counts())), columns= ['Flavor Attribution Counts']).reset_index()
flavor_df["DPT_DSC"] = flavor_df.SUBCOM_DSC.map(dict(zip(pim_load.SUBCOM_DSC, pim_load.DPT_DSC)))
flavor_df['Flavor Distribution'] = flavor_df.SUBCOM_DSC.map(dict(pim_load.groupby('SUBCOM_DSC').\
                                                                 apply(lambda x:dict( x.Flavor_Code.value_counts()))))
flavor_df['Item Count'] = flavor_df.SUBCOM_DSC.map(dict(pim_load.groupby('SUBCOM_DSC').size()))                                                        

In [105]:
flavor_df.to_csv('PIM_Flavor_Attributes_current.csv', index = None)

In [ ]:
flavor_df = pd.DataFrame(pim_load.groupby('SUBCOM_DSC').apply(lambda x:dict( x.Flavor_Flag.value_counts())), columns= ['Flavor Attribution Counts']).reset_index()
